In [4]:
#Membrane simulation of a cyclic peptide generated from Ohue Lab's cyclic peptide binder hallucination protocol.
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


In [1]:
!conda install -c "omnia/label/dev" openmmforcefields

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [2]:
!conda install -c conda-forge rdkit

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2



# All r

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/MyDrive

/content/drive/MyDrive


In [5]:
!pip install openmm

In [11]:
import openmm.app
from openmm.app import ForceField, Modeller, PME, Simulation, PDBFile, PDBReporter, StateDataReporter
from openmm import unit
import mdtraj as md
import openmm as mm
import numpy as np
import sys

In [15]:
!conda install -c conda-forge pdbfixer

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during cond

In [6]:
import pdbfixer
def prepare_protein(
    pdb_path, ignore_missing_residues=False, ignore_terminal_missing_residues=False, ph=7.0):
    """
    Use pdbfixer to prepare the protein from a PDB file. Hetero atoms such as ligands are
    removed and non-standard residues replaced. Missing atoms to existing residues are added.
    Missing residues are ignored by default, but can be included.
    Parameters
    ----------
    pdb_file: pathlib.Path or str
        PDB file containing the system to simulate.
    ignore_missing_residues: bool, optional
        If missing residues should be ignored or built.
    ignore_terminal_missing_residues: bool, optional
        If missing residues at the beginning and the end of a chain should be ignored or built.
    ph: float, optional
        pH value used to determine protonation state of residues
    Returns
    -------
    fixer: pdbfixer.pdbfixer.PDBFixer
        Prepared protein system.
    """

    pdb_file=str(pdb_path)
    fixer = pdbfixer.PDBFixer(str(pdb_file))
    fixer.removeHeterogens(keepWater=False)  # co-crystallized ligands are unknown to PDBFixer, and removing water
    fixer.findMissingResidues()  # identify missing residues, needed for identification of missing atoms

    # if missing terminal residues shall be ignored, remove them from the dictionary
    if ignore_terminal_missing_residues:
        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                del fixer.missingResidues[key]

    # if all missing residues shall be ignored ignored, clear the dictionary
    if ignore_missing_residues:
        fixer.missingResidues = {}


    fixer.findNonstandardResidues()  # find non-standard residue
    fixer.replaceNonstandardResidues()  # replace non-standard residues with standard one
    fixer.findMissingAtoms()  # find missing heavy atoms
    fixer.addMissingAtoms()  # add missing atoms and residues
    fixer.addMissingHydrogens(ph)  # add missing hydrogens
    return fixer


#prepare protein and build only missing non-terminal residues
prepared_protein = prepare_protein("binder.pdb", ignore_missing_residues=False, ph=7.0)

PDBFile.writeFile(prepared_protein.topology, prepared_protein.positions, open("binder_prepared.pdb", 'w'),keepIds=True)


In [12]:
# Load the molecule from the PDB file using OpenMM's Topology
pdb_topology = pdb.topology
modeller = Modeller(pdb_topology, pdb.positions)

# Load the force field and generate SMIRNOFF parameters
ff = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

# Add the membrane to the modeller

modeller.addMembrane(ff,
                     lipidType='POPC',
                      minimumPadding=1*unit.nanometer,
                    )

# Create the OpenMM system
system = ff.createSystem(modeller.topology, nonbondedMethod=PME)

# Set up the integrator and simulation
integrator = mm.LangevinIntegrator(
    300 * unit.kelvin, 1.0 / unit.picosecond, 2.0 * unit.femtoseconds
)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# Set up reporters
steps = 100
write_interval = 1
log_interval = 1

simulation.reporters.append(
    StateDataReporter(
        sys.stdout,
        log_interval,
        step=True,
        potentialEnergy=True,
        temperature=True,
        progress=True,
        remainingTime=True,
        speed=True,
        totalSteps=steps,
        separator="\t",
    )
)
simulation.reporters.append(PDBReporter('binder_md.pdb', 2))

simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(steps)

#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	1	-378109.1489471395	246.78831353871973	0	--
2.0%	2	-367298.58860881435	210.00072253073796	0.26	1:05
3.0%	3	-359047.11129019276	180.83748619153934	0.364	0:45
4.0%	4	-341080.1272169482	137.87206708740786	0.3	0:55
5.0%	5	-357096.988016136	173.92596512507532	0.342	0:47
6.0%	6	-354017.98373601754	169.35642622302788	0.279	0:58
7.0%	7	-346608.7828046546	152.35487479073214	0.282	0:57
8.0%	8	-356061.47704055853	170.89402116406734	0.226	1:10
9.0%	9	-342770.0363284563	141.90128025411042	0.235	1:06
10.0%	10	-349433.15575344523	158.36847789215972	0.214	1:12
11.0%	11	-357504.50235654693	178.26226691467144	0.219	1:10
12.0%	12	-345284.90012454276	152.5295146661096	0.198	1:16
13.0%	13	-354059.9630699999	169.9409689193764	0.203	1:13
14.0%	14	-347496.35060106963	154.25058912593033	0.192	1:17
15.0%	15	-341435.13548000035	141.1769916677214	0.199	1:13
16.0%	16	-354457.7364015955	170.77525616341956	0